In [163]:
import requests
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
tqdm.pandas()
with open('idioms_korean.txt', 'r') as file:
    idioms = file.read()

In [164]:
# Obtained from https://namu.wiki/w/%ED%95%9C%EA%B5%AD%EC%96%B4/%EA%B4%80%EC%9A%A9%EA%B5%AC
df = pd.read_fwf('idioms_korean.txt', header=None, names=['idiom'])
df.head()

,idiom
0,강 건너 불구경
1,가뭄에 콩 나듯
2,간(에) 바람 들다
3,간(에) 불 붙다
4,간(을) 빼 먹다


In [165]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Obtained from: https://en.dict.naver.com/#/main
def get_url(idiom):
    query = idiom.strip().replace(" ", "%20")
    return f"https://en.dict.naver.com/#/search?query={query}&range=word"

def get_match_and_meaning(url, xpaths):
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    for xpath in xpaths:
        try:
            element = driver.find_element(By.XPATH, xpath)
            xpaths[xpath] += 1
            text = element.text.split('\n')
            match = text[0]
            meaning = text[1]
            if meaning == "1.":
                meaning = text[2]
            return match, meaning
        except:
            continue
    return '', ''

def apply_selenium(row, xpaths):
    url = get_url(row['idiom'])
    row['match'], row['meaning'] = get_match_and_meaning(url, xpaths)
    return row

In [166]:
xpaths = {
    '//*[@id="searchPage_entry"]/div': 0,
    '//*[@id="searchPage_entry"]/div[0]': 0,
}
# subdf = df[:5].progress_apply(lambda row: apply_selenium(row, xpaths), axis=1)
df = df.progress_apply(lambda row: apply_selenium(row, xpaths), axis=1)

 97%|█████████▋| 396/408 [14:06<00:25,  2.10s/it]

In [ ]:
df.to_csv('idioms_korean.csv', index=False)